In [1]:
import os
MODEL_DIR = '../models/siamese'
LOG_DIR = '../logs/04/'
SIZE = (224, 224)
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
caminho_bases = os.path.join('..', 'bases', 'vazios')
caminho_train = os.path.join(caminho_bases, 'train')
caminho_test = os.path.join(caminho_bases, 'test')

# Callbacks

In [2]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard

tensorboard_logs = TensorBoard(log_dir=LOG_DIR, histogram_freq=1,
                               write_graph=False, write_images=False,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        '04modelweights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2,
                              verbose=1, min_delta=1e-2, mode='min')

# Model

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Activation, Conv2D, \
    BatchNormalization, concatenate, Dense, Dropout, Flatten, Input, MaxPooling2D


SIZE = (224, 224)


nuclear_model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3, 3),
                         padding='same',
                         activation='relu',
                         input_shape=(*SIZE, 3)),
  MaxPooling2D(pool_size=(2, 2)),
  Conv2D(32, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.2),
  Conv2D(64, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(128, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(256, (3, 3), activation='relu'),
  Flatten(),
  Dense(128, activation='relu'),
 
])

nuclear_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 28, 28, 64)        0

In [9]:
a_in = Input(shape=(*SIZE, 3))
b_in = Input(shape=(*SIZE, 3))

a_feat = nuclear_model(a_in)
b_feat = nuclear_model(b_in)

combined_features = concatenate([a_feat, b_feat], name = 'merge_features')
combined_features = Dense(16, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(4, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(1, activation = 'sigmoid')(combined_features)
similarity_model = tf.keras.Model(inputs = [a_in, b_in], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

similarity_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['mae'])

Model: "Similarity_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128)          1064608     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
merge_features (Concatenate)    (None, 256)          0           sequential[1][0]  

# Training

In [10]:
caminho_nvazio_train = os.path.join(caminho_train, 'nvazio')
caminho_vazio_train = os.path.join(caminho_train, 'vazio')
caminho_nvazio_test = os.path.join(caminho_test, 'nvazio')
caminho_vazio_test = os.path.join(caminho_test, 'vazio')


In [53]:
import random
import numpy as np
from PIL import Image

def generate_random_batch(nvazios_list, vazios_list, batch_size=32):
    def get_item(classe):
        if classe == 0:
            return nvazios_list.pop()
        return vazios_list.pop()
    result = []
    for i in range(batch_size):
        classe1 = random.randint(0, 1)
        item1 = get_item(classe1) 
        classe2 = random.randint(0, 1)
        item2 = get_item(classe2)
        if classe1 == classe2:
            if item1 == item2:
                alpha = 0
            else:
                alpha = 0.001
        else:
            alpha = 1
        result.append((item1, item2, alpha))
    return result  

def image_generator(caminho1, caminho2, batch_size=32):
    list_files1 = os.listdir(caminho1)
    list_files2 = os.listdir(caminho2)
    list1 = []
    list2 = []
    while True:
        # Ciclar lista (se acabar, recarregar do começo)
        if len(list1) < batch_size * 2:
            random.shuffle(list_files1)
            list1 = [os.path.join(caminho1, arq) for arq in list_files1]
        if len(list2) < batch_size * 2:
            random.shuffle(list_files2)
            list2 = [os.path.join(caminho2, arq) for arq in list_files2]
        # Gerar um batch da lista
        triples = generate_random_batch(list1, list2)
        X1 = np.zeros((batch_size, *SIZE, 3))
        X2 = np.zeros((batch_size, *SIZE, 3))
        y = np.zeros((batch_size, 1))
        '''for ind, triple in enumerate(triples):
            pil_img1 = Image.open(triple[0])
            pil_img1 = pil_img1.resize(SIZE, Image.ANTIALIAS)
            pil_img2 = Image.open(triple[1])
            pil_img2 = pil_img2.resize(SIZE, Image.ANTIALIAS)
            label = triple[2]
            X1[ind, :, :, :] = np.array(pil_img1) / 255.
            X2[ind, :, :, :] = np.array(pil_img2) / 255.
            y[ind, :] = label
        '''
        yield [X1, X2], y



In [54]:
train_generator = image_generator(caminho_nvazio_train, caminho_vazio_train)
validation_generator = image_generator(caminho_nvazio_test, caminho_vazio_test)

In [55]:
similarity_model.fit_generator(train_generator,
                               steps_per_epoch=200,
                               epochs=100,
                               verbose=1,
                               callbacks=[tensorboard_logs, mcp_save,
                                         early_stop, reduce_lr],
                               validation_data=validation_generator,
                               validation_steps=30)

Epoch 1/100
200/200 [==============================] - 40s 199ms/step - loss: 0.4244 - mae: 0.3112 - val_loss: 0.1121 - val_mae: 0.1060
Epoch 2/100
200/200 [==============================] - 40s 201ms/step - loss: 0.2356 - mae: 0.2025 - val_loss: 0.1991 - val_mae: 0.1805
Epoch 3/100
199/200 [============================>.] - ETA: 0s - loss: 0.1890 - mae: 0.1643
Epoch 00003: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
200/200 [==============================] - 39s 194ms/step - loss: 0.1890 - mae: 0.1642 - val_loss: 0.2678 - val_mae: 0.2349
Epoch 4/100
200/200 [==============================] - 39s 193ms/step - loss: 0.1697 - mae: 0.1480 - val_loss: 0.4010 - val_mae: 0.3304
Epoch 5/100
199/200 [============================>.] - ETA: 0s - loss: 0.1612 - mae: 0.1411
Epoch 00005: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
200/200 [==============================] - 39s 193ms/step - loss: 0.1611 - mae: 0.1410 - val_loss: 0.3973 - val_mae: 0.3279
